---
## Ejercicio 8

a) Desarrolle el método de la Transformada Inversa y el de Rechazo para generar una variable aleatoria
X cuya distribución de probabilidad está dada por:
$$
P(X=i) = \frac{\frac{\lambda^{i}}{i!}\cdot e^{-\lambda}}{\sum_{j=0}^k \frac{\lambda^j}{j!} \cdot e^{-\lambda}} \text{  }(i = 0,...,k)
$$

Primero que nada podemos observar que la FPM de X es una fracción donde en el numerador tenemos la FPM de una variable aleatoria Y con distribución de Poisson y abajo la sumatoria de Poisson truncada que más bien es una constante

In [5]:
#Importaciones
from math import exp
from random import random
from time import time

In [37]:
def Poisson(lamda:float) -> int:
    """
    Variable aleatoria con distribución de Poisson, sin optimizar

    Args:
        lamda (float): parámetro de la distribución

    Returns:
        int: valor obtenido (iteraciones)
    """
    U = random()
    i = 0
    p = exp(-lamda)
    F = p

    while U >= F:
        i += 1
        p *= lamda / i
        F = F + p

    return i


#FPM de Poisson
def Poisson_FPM(lamda: float, i: int) -> float:
    """
    P(Y=i)
    Es la función de masa de Poisson

    Args:
        lamda (float): parámetro de la distribución
        i (int): valor que toma la variable aleatoria

    Returns:
        float: probabilidad de que la variable aleatoria tome el valor i
    """
    probability = exp(-1*lamda)
    for j in range(1, i + 1):
        probability *= lamda / j
    return probability


#Constante de Poisson
def Poisson_sum(lamda: float, k: int) -> float:
    """
    Constante que va en el denominador
    Es la función de distribución acumulada de una Poisson cdf

    Args:
        lamda (float): parámetro de la distribución
        k (int): valor de truncamiento a la sumatoria

    Returns:
        float: valor acumulado hasta k de una Poisson
    """
    probability = exp(-1*lamda)
    summation = probability
    for j in range(1, k + 1):
        probability *= lamda / j
        summation += probability

    return summation


#Probabilidad del ejercicio
def PX_i(lamda:float, k:int, i:int) -> float:
    """
    P(X=i) con i=0,...,k

    Args:
        lamda (float): parámetro de la distribución
        k (int): valor de truncamiento para la sumatoria
        i (int): valor entero al que le calcularemos la probabilidad

    Returns:
        float: Probabilidad de que la v.a tome el valor i
    """
    return Poisson_FPM(lamda=lamda, i=i) / Poisson_sum(lamda=lamda, k=k)


b) Estime P(X > 2) con k = 10 y λ = 0,7, y 1000 repeticiones. Compare con el valor exacto.


In [38]:
#Simulación para valor exacto
def probability_simulation(X, Nsim:int, lamda:float, k:int, i:int) -> float:
    probability = 0
    for _ in range(Nsim):
        x = X(lamda=lamda, k=k)
        if x > i:
            probability += 1
    
    return probability / Nsim

In [45]:
#Transformada inversa
def Poisson_ITX(lamda:float, k:int) -> int:
    """
    Poisson con transformada inversa

    Args:
        lamda (float): parámetro de la distribución
        k (int): valor de truncamiento

    Returns:
        int: iteraciones
    """
    u = random()
    i = 0
    summation = Poisson_sum(lamda, k)
    p = exp(-1 * lamda) / summation
    F = p
    while u >= F:
        i += 1
        p *= lamda / i
        F = F + p
    return i

def Poisson_ITX_OPT(lamda: float, k:int) -> int:
    """
    Poisson con transformada inversa mejorada

    Args:
        lamda (float): parámetro de la distribución
        k (int): valor de truncamiento

    Returns:
        int: iteraciones
    """
    S = Poisson_sum(lamda, k)
    p = exp(-1 * lamda) / S 
    F = p
    for j in range(int(lamda)):
        p *= lamda / j
        F += p
    j = int(lamda)
    u = random()
    if u < F:
        while u < F:
            F -= p
            p *= j / lamda
            j -= 1
        return j + 1
    else:
        while u >= F:
            j += 1
            p *= lamda / j
            F += p
        return j


# ACEPTACION Y RECHAZO
def Poisson_AyR(lamda:float, k: int) -> float:
    """
    Generación de v.a de Poisson por el método de aceptación y rechazo

    Args:
        lamda (float): parámetro de la distribución
        k (int): valor de truncamiento

    Returns:
        float: valor generado por la v.a
    """
    y = Poisson(lamda=lamda)
    u = random()
    summation_constant = Poisson_sum(lamda=lamda, k=k)
    c = 1 / summation_constant
    qy = Poisson_FPM(lamda=lamda, i=y)
    while u >= PX_i(lamda=lamda, k=k, i=y) / (c * qy):
        y = Poisson(lamba=lamda)
        qy = Poisson_FPM(lamda=lamda, i=y)
        u = random()
    return y

def Poisson_AyR_OPT(lamda:float, k:int) -> float:
    """
        Generación de v.a de Poisson por el método de aceptación y rechazo optimizado

    Args:
        lamda (float): parámetro de la distribución
        k (int): valor de truncamiento

    Returns:
        float: valor generado por la v.a
    """
    while True:
        y = Poisson(lamda=lamda)
        if y <= k:
            return y

In [49]:
print("Estimación de P(X>2) con 1000 simulaciones")
print(f"T. Inversa: {probability_simulation(X=Poisson_ITX,Nsim=1000, lamda=0.7, k=10, i=2)}")
print(f"T. Inversa MEJORADA: {probability_simulation(X=Poisson_ITX_OPT, Nsim=1000, lamda=0.7, k=10, i=2)}" )
print("----------------------------------------------------------------------------------------")
print(f"Estimación usando Aceptación y Rechazo: {probability_simulation(X=Poisson_AyR, Nsim=1000, lamda=0.7, k=10, i=2)}")
print(f"Método Mejorado: {probability_simulation(X=Poisson_AyR_OPT, Nsim=1000, lamda=0.7, k=0, i=2)}")
print("----------------------------------------------------------------------------------------")
print(f"Valor exacto P(X > 2) = {1 - (PX_i(lamda=0.7, k=10, i=0) + PX_i(lamda=0.7, k=10, i=1) + PX_i(lamda=0.7, k=10, i=2))}")

# Si quiero comparar el tiempo de corrida de las distintas funciones de Transf Inversa
print("----------------------------------------------------------------------------------------")
start = time()
probability_simulation(Poisson_ITX, lamda=0.7, k=10, i=2, Nsim=10000)
print(f"Tiempo de corrida comun: {time() - start:.5f}")
start = time()
probability_simulation(Poisson_ITX_OPT, lamda=0.7, k=10, i=2, Nsim=10000)
print(f"Tiempo de corrida mejorada: {time() - start:.5f}")
start = time()
probability_simulation(Poisson_AyR, lamda=0.7, k=10, i=2, Nsim=10000)
print(f"Tiempo de corrida AyR: {time() - start:.5f}")
start = time()
probability_simulation(Poisson_AyR_OPT, lamda=0.7, k=10, i=2, Nsim=10000)
print(f"Tiempo de corrida AyR mejorada: {time() - start:.5f}")

Estimación de P(X>2) con 1000 simulaciones
T. Inversa: 0.034
T. Inversa MEJORADA: 0.033
----------------------------------------------------------------------------------------
Estimación usando Aceptación y Rechazo: 0.034
Método Mejorado: 0.0
----------------------------------------------------------------------------------------
Valor exacto P(X > 2) = 0.03414158387347266
----------------------------------------------------------------------------------------
Tiempo de corrida comun: 0.01233
Tiempo de corrida mejorada: 0.01385
Tiempo de corrida AyR: 0.02860
Tiempo de corrida AyR mejorada: 0.00319
